In [1]:
# https://wikidocs.net/63565 참고

In [2]:
import torch
import torch.nn as nn

## CNN의 두가지 표기 방법
1. 합성곱(nn.Conv2d) + 활성화 함수(nn.ReLU)를 하나의 합성곱 층으로 보고,맥스풀링(nn.MaxPoold2d)은 풀링 층으로 별도로 명명
2. 합성곱(nn.Conv2d) + 활성화 함수(nn.ReLU) + 맥스풀링(nn.MaxPoold2d)을 하나의 합성곱 층으로 명명

## layer 3개 짜리 Model Architecture
- 1번 레이어 : 합성곱층, Convolution layer
    - 합성곱(in_channel=1, out_channel=32, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
    - 맥스풀링(kernel_size=2, stride=2)
- 2번 레이어 : 합성곱층, Convolution layer
    - 합성곱(in_channel=32, out_channel=64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
    - 맥스풀링(kernel_size=2, stride=2)
- 3번 레이어 : 전결합층, Fully-Connected layer
    - feature map을 펼침 : batch_size x 7 x 7 x 64 -> batch_size x 3136
    - 전결합층 (뉴런 10개) + 활성화 함수 Softmax

# declare

In [3]:
inputs = torch.Tensor(1,1,28,28)
# 1 x 1 x 28 x 28 텐서 생성
# (배치 크기 x 채널 x 높이(height) x 너비(width))
inputs.shape

torch.Size([1, 1, 28, 28])

In [4]:
conv1 = nn.Conv2d(1,32,3,padding=1)
# 1채널 짜리 입력 받아서, 32채널을 출력, 커널 사이즈는 3, 패딩은 1
conv1

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [5]:
conv2 = nn.Conv2d(32,64,kernel_size=3,padding=1)
# 32채널 짜리 입력 받아서, 64
conv2

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [6]:
pool = nn.MaxPool2d(2)
# 커널 사이즈와 스트라이드 둘다 2로 지정
pool

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

# make model

In [7]:
## layer 1

out = conv1(inputs)
# inputs를 첫번째 convolution에 통과
# 커널 사이즈 3x3에 패딩 1이면 크기가 보존됨
out.shape

torch.Size([1, 32, 28, 28])

In [8]:
out = pool(out)
out.shape

torch.Size([1, 32, 14, 14])

In [9]:
## layer 2

out = conv2(out)
out.shape

torch.Size([1, 64, 14, 14])

In [10]:
out = pool(out)
out.shape

torch.Size([1, 64, 7, 7])

In [11]:
## layer 3

out.size(0)

1

In [12]:
out.size(1)

64

In [13]:
out.size(2)

7

In [14]:
out.size(3)

7

In [15]:
out = out.view(out.size(0), -1)
# 첫번째 차원인 batch 차원은 놔두고, 나머지는 펼쳐라
# 제외하고 나머지는 하나의 차원으로 통합
out.shape

torch.Size([1, 3136])

In [16]:
fc = nn.Linear(3136, 10)
# input_dim = 3136, output_dim = 10
# 10개 차원의 텐서로 변환
out = fc(out)
out.shape

torch.Size([1, 10])

# MNIST 분류

In [17]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [18]:
# gpu 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

device

'cuda'

In [19]:
# 학습 파라미터 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [20]:
# 데이터셋 정의
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

In [21]:
# 데이터로더로 배치 크기를 지정
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [22]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

In [23]:
model = CNN().to(device)

In [24]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [25]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


In [26]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.225609481
[Epoch:    2] cost = 0.062334951
[Epoch:    3] cost = 0.0453871973
[Epoch:    4] cost = 0.0368103869
[Epoch:    5] cost = 0.0306432135
[Epoch:    6] cost = 0.0255049877
[Epoch:    7] cost = 0.0216905233
[Epoch:    8] cost = 0.0180997383
[Epoch:    9] cost = 0.0160109308
[Epoch:   10] cost = 0.0130720558
[Epoch:   11] cost = 0.0102106333
[Epoch:   12] cost = 0.00972270034
[Epoch:   13] cost = 0.00894099288
[Epoch:   14] cost = 0.00622102804
[Epoch:   15] cost = 0.0063359323


In [27]:
# 학습을 진행하지 않을 거라서 no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9884999990463257


/home/sangho/anaconda3/envs/tf/lib/python3.7/site-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/home/sangho/anaconda3/envs/tf/lib/python3.7/site-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


# layer 더 쌓기

## layer 5개 짜리 Model Architecture
- 1번 레이어 : 합성곱층, Convolution layer
    - 합성곱(in_channel=1, out_channel=32, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
    - 맥스풀링(kernel_size=2, stride=2)
- 2번 레이어 : 합성곱층, Convolution layer
    - 합성곱(in_channel=32, out_channel=64, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
    - 맥스풀링(kernel_size=2, stride=2)
- 3번 레이어 : 합성곱층, Convolution layer
    - 합성곱(in_channel=64, out_channel=128, kernel_size=3, stride=1, padding=1) + 활성화 함수 ReLU
    - 맥스풀링(kernel_size=2, stride=2, padding=1)
- 4번 레이어 : 전결합층, Fully-Connected layer
    - feature map을 펼침 : batch_size x 4 x 4 x 128 -> batch_size x 2048
    - 전결합층 (뉴런 625개) + 활성화 함수 ReLU
- 5번 레이어 : 전결합층, Fully-Connected layer
    - 전결합층 (뉴런 10개) + 활성화 함수 Softmax

In [28]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [29]:
# gpu 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

device

'cuda'

In [30]:
# 학습 파라미터 설정
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [31]:
# 데이터셋 정의
mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

In [32]:
# 데이터로더로 배치 크기를 지정
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [33]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L3 ImgIn shape=(?, 7, 7, 64)
        #    Conv      ->(?, 7, 7, 128)
        #    Pool      ->(?, 4, 4, 128)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))

        # L4 FC 4x4x128 inputs -> 625 outputs
        self.fc1 = torch.nn.Linear(4 * 4 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))
        # L5 Final FC 625 inputs -> 10 outputs
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [34]:
model = CNN().to(device)

In [35]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [36]:
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

총 배치의 수 : 600


In [37]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.188949957
[Epoch:    2] cost = 0.0517618656
[Epoch:    3] cost = 0.0364419185
[Epoch:    4] cost = 0.0299992282
[Epoch:    5] cost = 0.0232932996
[Epoch:    6] cost = 0.0189718064
[Epoch:    7] cost = 0.0175555684
[Epoch:    8] cost = 0.0142452996
[Epoch:    9] cost = 0.0141980378
[Epoch:   10] cost = 0.0124818077
[Epoch:   11] cost = 0.00869015511
[Epoch:   12] cost = 0.0123938946
[Epoch:   13] cost = 0.00819434412
[Epoch:   14] cost = 0.00656405883
[Epoch:   15] cost = 0.00754712848


In [38]:
# 학습을 진행하지 않을 거라서 no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9825999736785889
